# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
! pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6ac7cabe379d4b8b9f866bd782867162e9257f052497fc74faa8db837da4b74b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
!pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=3da2371bfadaecdccd8632306ace0098b00909f3d0568229b43e3449102e9829
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
Successfully built keybert


Lendo o Arquivo

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [6]:
num_gen = 5
perc = 5
temperatura = 0.75
tamanho = 45
i=1

In [7]:
arq = '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [8]:
base_cru_1p.head()

,texto,label
0,@user @user @user I’m really not on any side o...,1
1,@user @user Not all of us Conservatives are th...,0
2,Here is an Awesome Video of Alex Jones invadin...,0
3,@user Bs. He is looking for retirement money a...,0
4,@user @user @user Oh boy just wait! YOU ARE IN...,1


In [9]:
n_labels = base_cru_1p.label.nunique()
n_labels

2

In [10]:
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(base_cru_1p)

In [11]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

In [12]:
train_dataset = sample_dataset(train, label_column="label", num_samples=16)

In [13]:
train_dataset

Dataset({
    features: ['texto', 'label'],
    num_rows: 32
})

In [14]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')

In [15]:
classifica = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    # use_differentiable_head=True,
    # head_params={"out_features": 6},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [16]:
trainer = SetFitTrainer(
    model=classifica,
    train_dataset=train_dataset,
    # eval_dataset=eval,
    loss_class=CosineSimilarityLoss,
    # metric=f1_multiclass,
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning 40
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"texto": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

In [17]:
trainer.train()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1280
  Num epochs = 1
  Total optimization steps = 80
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

In [18]:
pred = classifica(base_cru_1p.texto.values)

In [19]:
f1_multiclass(base_cru_1p.label.values, pred.tolist())

0.7523310023310024

In [20]:
trainer.model._save_pretrained(save_directory="./setfit_output/")

In [21]:
def retorna_word(lista_palavras):
  res = []
  for a in lista_palavras:
    res.append(a[0])
  return res

In [22]:

from keybert import KeyBERT
kw_model = KeyBERT(model = "./setfit_output/")


In [23]:
base_cru_1p['keys'] = base_cru_1p['texto'].progress_apply(lambda x:retorna_word(kw_model.extract_keywords(x, keyphrase_ngram_range=(1, 3), top_n=3, use_mmr=True, stop_words = None)))

100%|██████████| 170/170 [00:15<00:00, 11.25it/s]


In [24]:
base_cru_1p.head()

,texto,label,keys
0,@user @user @user I’m really not on any side o...,1,"[shit jsjdhkdfhjfkfkhsjdhd messsssss, talking ..."
1,@user @user Not all of us Conservatives are th...,0,"[sickens me he, spewing us propaganda, gullibl..."
2,Here is an Awesome Video of Alex Jones invadin...,0,"[vaxxed readthebook cdcwhistleblower, awesome ..."
3,@user Bs. He is looking for retirement money a...,0,"[bs he is, new job, retirement money]"
4,@user @user @user Oh boy just wait! YOU ARE IN...,1,"[be that ignorant, oh boy just, in for surprise]"


In [25]:
base_cru_1p['keys_text'] = base_cru_1p['keys'].progress_apply(lambda x: " - ".join(x))
base_cru_1p.head()

100%|██████████| 170/170 [00:00<00:00, 45987.21it/s]


,texto,label,keys,keys_text
0,@user @user @user I’m really not on any side o...,1,"[shit jsjdhkdfhjfkfkhsjdhd messsssss, talking ...",shit jsjdhkdfhjfkfkhsjdhd messsssss - talking ...
1,@user @user Not all of us Conservatives are th...,0,"[sickens me he, spewing us propaganda, gullibl...",sickens me he - spewing us propaganda - gullib...
2,Here is an Awesome Video of Alex Jones invadin...,0,"[vaxxed readthebook cdcwhistleblower, awesome ...",vaxxed readthebook cdcwhistleblower - awesome ...
3,@user Bs. He is looking for retirement money a...,0,"[bs he is, new job, retirement money]",bs he is - new job - retirement money
4,@user @user @user Oh boy just wait! YOU ARE IN...,1,"[be that ignorant, oh boy just, in for surprise]",be that ignorant - oh boy just - in for surprise


In [26]:
pos_key = {}
for c in base_cru_1p['label'].unique():
  data = base_cru_1p[base_cru_1p['label']==c].copy()
  original = data[['keys_text', 'texto', 'label']].values.tolist()

  pos_key[c] = original

  print(len(pos_key[c]))

56
114


In [27]:
from random import sample

examples = []
generate = []
original = []
labels = []

for c in base_cru_1p['label'].unique():
  for n in range(int(2*num_gen*len(pos_key[c]))):
    if len(pos_key[c]) > 11:
      amostra = sample(pos_key[c],11)
    else:
      amostra = sample(pos_key[c],len(pos_key[c]))

    gen = amostra.pop()

    examples.append(amostra)
    generate.append(gen[0])
    original.append(gen[1])
    labels.append(gen[2])


data_fim = {'examples':examples, 'generate':generate, 'original':original, 'labels':labels}

base_gen = pd.DataFrame.from_dict(data_fim)
base_gen.head()

,examples,generate,original,labels
0,"[[he mess - good he mess - user good, @user Go...",is disgusting statement - this is disgusting -...,.@USER is staffed with unhinged Liberals &amp;...,1
1,"[[slamming chris hayes - love rapist - putin, ...",he is unfit - that stopkavanagh repeatedly - t...,@user Just the fact that #StopKavanagh REPEATE...,1
2,[[big dick wrong - spelled big dick - dick wro...,boys who hang - avoid the freshman - hit on me,@user me trying to avoid the freshman boys who...,1
3,[[drunk as shit - was definitely drunk - user ...,user bad bitch - bad bitch now - now lol,@user U a bad bitch now lol,1
4,[[an evil hateful - he real racist - amp obama...,shitty ghetto overrun - user dishonored - rats...,@user Dishonored zone where you have to balanc...,1


In [28]:
base_gen.shape

(1700, 4)

In [29]:
def cria_prompt(classe, ex, synt):
  prompt = "Generate a tweet with this few keywords:"
  for el in ex:
    soma = '\nkeys: '+ el[0]+'\ntweet: '+el[1]+'\n###'
    prompt+= soma

  prompt += '\nkeys: '+ synt+'\ntweet: '
  return prompt



In [30]:
dict_label ={0:'non-offensive', 1:'offensive'}
base_gen['class_name']= base_gen['labels'].map(dict_label)
base_gen.head()

,examples,generate,original,labels,class_name
0,"[[he mess - good he mess - user good, @user Go...",is disgusting statement - this is disgusting -...,.@USER is staffed with unhinged Liberals &amp;...,1,offensive
1,"[[slamming chris hayes - love rapist - putin, ...",he is unfit - that stopkavanagh repeatedly - t...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive
2,[[big dick wrong - spelled big dick - dick wro...,boys who hang - avoid the freshman - hit on me,@user me trying to avoid the freshman boys who...,1,offensive
3,[[drunk as shit - was definitely drunk - user ...,user bad bitch - bad bitch now - now lol,@user U a bad bitch now lol,1,offensive
4,[[an evil hateful - he real racist - amp obama...,shitty ghetto overrun - user dishonored - rats...,@user Dishonored zone where you have to balanc...,1,offensive


In [31]:
base_gen['prompt'] = base_gen.progress_apply(lambda x: cria_prompt(x.class_name, x.examples, x.generate), axis=1)
base_gen.head()

100%|██████████| 1700/1700 [00:00<00:00, 27115.39it/s]


,examples,generate,original,labels,class_name,prompt
0,"[[he mess - good he mess - user good, @user Go...",is disgusting statement - this is disgusting -...,.@USER is staffed with unhinged Liberals &amp;...,1,offensive,Generate a tweet with this few keywords:\nkeys...
1,"[[slamming chris hayes - love rapist - putin, ...",he is unfit - that stopkavanagh repeatedly - t...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive,Generate a tweet with this few keywords:\nkeys...
2,[[big dick wrong - spelled big dick - dick wro...,boys who hang - avoid the freshman - hit on me,@user me trying to avoid the freshman boys who...,1,offensive,Generate a tweet with this few keywords:\nkeys...
3,[[drunk as shit - was definitely drunk - user ...,user bad bitch - bad bitch now - now lol,@user U a bad bitch now lol,1,offensive,Generate a tweet with this few keywords:\nkeys...
4,[[an evil hateful - he real racist - amp obama...,shitty ghetto overrun - user dishonored - rats...,@user Dishonored zone where you have to balanc...,1,offensive,Generate a tweet with this few keywords:\nkeys...


In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda:0"

model_id = "EleutherAI/gpt-neo-2.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
# model_4bit = model_4bit.to(device)

In [33]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [34]:

print(gera_text(model_4bit, tokenizer, device,base_gen['prompt'].iloc[4], temperatura, tamanho))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generate a tweet with this few keywords:
keys: an evil hateful - he real racist - amp obama
tweet: @user Holder is an evil, hateful, dangerous man.  He's a real"" racist.  He absolutely hates White People.  He &amp; Obama contributed to the race riots during O's Admin.""
###
keys: really sexually harassed - block nomination liberals - really don
tweet: @user @user This is purely a move to block a nomination. Liberals said early on they would not vote for him. This made up story is a slap in the face to all who were really sexually harassed. It shows liberals really don’t care about any true victims. Power first.
###
keys: slips don want - trail of corruption - they really are
tweet: WHY... it's their trail of corruption... and they have so much of it going on, and at all times, they don't want any slips""... don't want American people to know who they really are AND WHAT THEY'VE BEEN UP TO... #MAGA
###
keys: fucking boring and - it so fucking - sexually active we
tweet: @user Still bet

In [35]:
device = "cuda:0"
base_gen['gpt_resp'] = base_gen['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, temperatura, tamanho))

100%|██████████| 1700/1700 [2:16:37<00:00,  4.82s/it]


In [36]:
base_gen.head()

,examples,generate,original,labels,class_name,prompt,gpt_resp
0,"[[he mess - good he mess - user good, @user Go...",is disgusting statement - this is disgusting -...,.@USER is staffed with unhinged Liberals &amp;...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
1,"[[slamming chris hayes - love rapist - putin, ...",he is unfit - that stopkavanagh repeatedly - t...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
2,[[big dick wrong - spelled big dick - dick wro...,boys who hang - avoid the freshman - hit on me,@user me trying to avoid the freshman boys who...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
3,[[drunk as shit - was definitely drunk - user ...,user bad bitch - bad bitch now - now lol,@user U a bad bitch now lol,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
4,[[an evil hateful - he real racist - amp obama...,shitty ghetto overrun - user dishonored - rats...,@user Dishonored zone where you have to balanc...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...


In [37]:
def separa_resp(gerar, resp):
  fim = resp.split(gerar+'\ntweet:')[1]
  return fim.split('###')[0].replace('\n', '')


In [38]:
base_gen['gen_fim'] = base_gen.progress_apply(lambda x: separa_resp(x.generate, x.gpt_resp), axis=1)
base_gen.head()

100%|██████████| 1700/1700 [00:00<00:00, 26993.03it/s]


,examples,generate,original,labels,class_name,prompt,gpt_resp,gen_fim
0,"[[he mess - good he mess - user good, @user Go...",is disgusting statement - this is disgusting -...,.@USER is staffed with unhinged Liberals &amp;...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user The statement is disgusting. It doesn'...
1,"[[slamming chris hayes - love rapist - putin, ...",he is unfit - that stopkavanagh repeatedly - t...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user @user @user he is unfit to be presiden...
2,[[big dick wrong - spelled big dick - dick wro...,boys who hang - avoid the freshman - hit on me,@user me trying to avoid the freshman boys who...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,????????? Fuck me if I hit on the freshman!?!?...
3,[[drunk as shit - was definitely drunk - user ...,user bad bitch - bad bitch now - now lol,@user U a bad bitch now lol,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,????? I'm a bitch??????
4,[[an evil hateful - he real racist - amp obama...,shitty ghetto overrun - user dishonored - rats...,@user Dishonored zone where you have to balanc...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user @user @user @user @user @user @user @u...


In [39]:
teste = base_gen[base_gen['gen_fim']!= ' '].copy()
teste.shape

(1669, 8)

In [40]:
exp_lambada = []

limiar = 1.2/n_labels
base_gen = base_gen[~(base_gen['gen_fim'].isin(["ERRO",'',' ', '\n']))].copy()
preds = classifica.predict_proba(base_gen['gen_fim'].to_list()).tolist()
base_gen['preds'] = [preds[i] for i in range(len(preds))]
base_gen['prob_text'] = base_gen.progress_apply(lambda x:(x.preds[x.labels]), axis=1)
# fim = base_gen[base_gen['prob_text']>= limiar].copy()
fim = base_gen.copy()
fim.rename(columns = {'original':'texto', 'gen_fim':'generated', 'labels':'new_label'}, inplace = True)


exp_lambada.append(fim[['texto','generated', 'new_label', 'prob_text']].copy())

100%|██████████| 1669/1669 [00:00<00:00, 39556.61it/s]


In [41]:
exp_lambada[0].shape

(1669, 4)

In [42]:
base_cru_1p.shape

(170, 4)

In [43]:
exp_lambada[0].shape[0]/base_cru_1p.shape[0]

9.81764705882353

In [44]:
fim = []

for c in base_cru_1p.label.unique():
  limite = base_cru_1p[base_cru_1p.label==c].copy().shape[0]*num_gen
  gerados = exp_lambada[0][exp_lambada[0]['new_label']==c].copy()
  # gerados = gerados.drop_duplicates(subset = ['generated'])

  gerados = gerados.sort_values(by='prob_text', ascending=False)
  gerados = gerados.head(limite)
  fim.append(gerados)

In [45]:
base_aug_fim = pd.concat(fim)
base_aug_fim.head()

,texto,generated,new_label,prob_text
68,@user @user @user @user He's having a go at pe...,I think that roger waters is a bit of a dick...,1,0.985818
544,@user she is uncultured,She's uncultured &amp; a total douche bag,1,0.984464
301,@user Yeahhhhh I will admit to being a Destiny...,@user @user @user I've made him suck in a cu...,1,0.984102
207,@user I don’t like her but hope if she is next...,@user @user Yeah. I don't like her. She's a ...,1,0.982740
231,@user @user lmao @ the WORLD league viewership...,@user he is on console and we are watching t...,1,0.982581


In [46]:
base_aug_fim.shape[0]/base_cru_1p.shape[0]

5.0

In [47]:
arquivo = '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/tamanho/base_offensive_tamanho_'+str(tamanho)+'.csv'
base_aug_fim.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')